# UK used Cars - Car Price Prediction Regression Model
--- 

In [2]:
## EDA Standard Libary

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats as ss

In [3]:
import os
#Define ROOT_DIR as the root directory of the project on your local machine
ROOT_DIS = os.path.realpath(os.path.join(os.path.dirname('1-Data Preparation.ipynb'), '..'))
ROOT_DIS


'/Users/Dwika/My Projects/FINAL PROJECT PURWADHIKA'

## Stakeholder Statements
---

###     Business Problem
Online used car listing company (e.g: OLX Autos, Carmudi, Carsome.id) 

### Business Model: 
Mereka beli mobil bekas dengan kondisi yang bagus harga dibawah pasar, untuk dijual lagi di platform mereka dengan harga yang lebih tinggi

### Business Problem:
- Jika membeli mobil harganya ketinggian maka profitnya lebih kecil

### Business Success Criteria:
- Mereka ingin mengetahui range harga mobil bekas yang akan mereka beli, agar mereka bisa menentukan harga jual yang tepat
- Tetap memperhitungkan profit dalam hasil prediksi harga mobil bekas

### 
Solution:
- Membuat model machine learning untuk memprediksi harga mobil bekas, sehingga ketika agen pembeli mobil bekas menemukan mobil bekas yang akan dibeli, mereka bisa mengetahui range harga beli yang tepat, sehingga profit yang didapat maksimal

In [ ]:
dfdsfds

print('hello)